In [1]:
import numpy as np 
import pandas as pd
from shapely.geometry import shape, Point

import multiprocessing
from IPython.display import clear_output

In [2]:
trip_data = pd.read_csv("data/data.csv")
cologne_districts = pd.read_json("data/cologne.json")
hexagons = pd.read_json("data/hexagons/hexagons_res8.json")

In [3]:
#MULTIPROCESSING
num_processes = multiprocessing.cpu_count() - 1
chunk_size = int(trip_data.shape[0]/num_processes)

chunks = [trip_data.iloc[i:i + chunk_size,:] for i in range(0, trip_data.shape[0], chunk_size)]

In [4]:
district_polygons = []
partition_polygons = []
dfLength = len(trip_data)

for feature in cologne_districts["features"]:
    hexagon = shape(feature["geometry"])
    id = feature["properties"]["name"]

    district_polygons.append({"id": id, "polygon": hexagon})


for feature in hexagons["features"]:
    polygon = shape(feature["geometry"])
    id = feature["id"] # for vor: ["properties"]["index"] for districts: ["properties"]["cartodb_id"] TODO: streamline

    partition_polygons.append({"id": id, "polygon": hexagon})

df_districts = pd.DataFrame(district_polygons)
df_polygons = pd.DataFrame(partition_polygons)

In [5]:
def map_polygons(trip_data_chunks):
    for index, row in enumerate(trip_data_chunks.itertuples()):
        starting_point = Point(row[6], row[5])
        ending_point = Point(row[8], row[7])

        for polygon in df_districts.itertuples():

            if polygon[2].contains(starting_point):
                trip_data_chunks.at[row.Index, "start_district"] = polygon[1]

            if polygon[2].contains(ending_point):
                trip_data_chunks.at[row.Index, "end_district"] = polygon[1]

        for hexagon in df_polygons.itertuples():

            if hexagon[2].contains(starting_point):
                trip_data_chunks.at[row.Index, "start_polygon"] = hexagon[1]
            else:
                trip_data_chunks.at[row.Index, "start_polygon"] = "null"


            if hexagon[2].contains(ending_point):
                trip_data_chunks.at[row.Index, "end_polygon"] = hexagon[1]
            else:
                trip_data_chunks.at[row.Index, "end_polygon"] = "null"

In [ ]:
pool = multiprocessing.Pool(processes=num_processes)

result = pool.map(map_polygons, chunks)

for i in range(len(result)):
   # since result[i] is just a dataframe
   # we can reassign the original dataframe based on the index of each chunk
   trip_data.iloc[result[i].index] = result[i]

print(trip_data)